In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
x_train = x_train[:,:,:,0]
x_test = x_test[:,:,:,0]

In [4]:
x_val = x_test[:7000]
x_test = x_test[7000:]
print("Validation Data Shape: {0} \nTest Data Shape: {1}".format(x_val.shape, x_test.shape))

Validation Data Shape: (7000, 32, 32) 
Test Data Shape: (3000, 32, 32)


In [5]:
input_img = Input(shape=(32, 32))

#Encoding
x = Dense(128, activation = 'relu')(input_img)
x = Dense(64, activation = 'relu')(x)
x = Dense(32, activation = 'relu')(x)
x = Dense(16, activation = 'relu')(x)
encoded = Dense(8, activation = 'relu')(x)

# Decoding
x = Dense(8, activation = 'relu')(encoded)
x = Dense(16, activation = 'relu')(x)
x = Dense(32, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(128, activation = 'relu')(x)
decoded = Dense(32, activation = 'sigmoid')(x)

In [6]:
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
batch_size = 32
num_classes = 10
epochs = 100
es = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1, mode = 'auto')
chkpt = 'AutoEncoder_Cifar10_Deep_weights.hdf5'
mc = ModelCheckpoint(filepath = chkpt, monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'auto')

history = model.fit(x_train, x_train, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_val, x_val), 
                    callbacks = [es, mc], shuffle = True)

Epoch 1/100
1556/1563 [============================>.] - ETA: 0s - loss: 0.5871
Epoch 00001: val_loss improved from inf to 0.57992, saving model to AutoEncoder_Cifar10_Deep_weights.hdf5
1563/1563 [==============================] - 12s 8ms/step - loss: 0.5870 - val_loss: 0.5799
Epoch 2/100
1554/1563 [============================>.] - ETA: 0s - loss: 0.5783
Epoch 00002: val_loss improved from 0.57992 to 0.57868, saving model to AutoEncoder_Cifar10_Deep_weights.hdf5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5783 - val_loss: 0.5787
Epoch 3/100
1556/1563 [============================>.] - ETA: 0s - loss: 0.5776
Epoch 00003: val_loss improved from 0.57868 to 0.57473, saving model to AutoEncoder_Cifar10_Deep_weights.hdf5
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5776 - val_loss: 0.5747
Epoch 4/100
1563/1563 [==============================] - ETA: 0s - loss: 0.5726
Epoch 00004: val_loss improved from 0.57473 to 0.57308, saving model to AutoEn

In [ ]:
score = model.evaluate(x_test, x_test, verbose=1)
print(score)

In [ ]:
c10test = model.predict(x_test)
c10val = model.predict(x_val)
print("Validation Dataset Score: {0}\nTest Dataset Score: {1}".format(np.average(c10val), np.average(c10test)))

In [ ]:
def showOrigDec(orig, dec, num=10):
    n = num 
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(orig[i].reshape(32,32))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(dec[i].reshape(32,32))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
showOrigDec(x_test, c10test)

In [ ]:
showOrigDec(x_test[100:], c10test[100:])

In [ ]:
showOrigDec(x_test[200:], c10test[200:])